In [1]:
# to construct byte-level base tokenizer
from magicab.etoken import TokenTrie 
from magicab import ETokenizer 

tokenizer = ETokenizer(mode="byte")

In [2]:
import os
os.cpu_count()

10

In [4]:
from magicab.etoken import TokenTrie 
from magicab import ETokenizer 
import time
import warnings
import asyncio 
from nest_asyncio import apply
apply()

tokenizer = ETokenizer(mode="byte")
texts = ["I am super duper"*1000, "Hey how is it going?"*1000, "I am not super duper"*1000] * 1000

start = time.time()
results = asyncio.run(tokenizer.encode_with_chunking(texts, batch_size=1000, mode='parallel'))
end = time.time()
time_parallel = end - start
print(f"Time for encoding (parallel): {time_parallel} seconds")


start = time.time()
results = tokenizer.encode_with_chunking(texts, batch_size=1000, mode='sequential')
end = time.time()
time_sequential = end - start
print(f"Time for encoding (sequential): {time_sequential} seconds")

Time for encoding (parallel): 1.8835067749023438e-05 seconds


In [10]:
from multiprocessing import Pool

# def f(x):
#     return x*x

# if __name__ == '__main__':
#     with Pool(5) as p:
#         print(p.map(f, [1, 2, 3]))

texts = ["I am super duper"*1000, "Hey how is it going?"*1000, "I am not super duper"*1000] * 1000


# sequential encoding
from magicab.etoken import chunk_text, _encode_chunks
import time 

chunk_size = 256 

def _single_encoding(text): 
    chunks = chunk_text(text, chunk_size)
    return _encode_chunks(chunks, tokenizer, chunk_size)

def sequential_encoding(texts):
    ids = []
    for text in texts: 
        ids.extend(_single_encoding(text))
    return ids

def multiprocessing_encoding(texts):
    with Pool(5) as p:
        return p.map(_single_encoding, texts)

multiprocessing_encoding(texts)

Process SpawnPoolWorker-9:
Process SpawnPoolWorker-10:
Process SpawnPoolWorker-11:
Process SpawnPoolWorker-12:
Process SpawnPoolWorker-13:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/anaconda3/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute '_single_encoding' on <module '__main__' (built-in)>
  File "/opt/homebrew/anaconda3/lib/python3.11/m

KeyboardInterrupt: 

In [5]:
type(results) # should be list of list(int) output

coroutine

In [7]:
results[0]

TypeError: 'coroutine' object is not subscriptable

In [14]:
start = time.time()
tokenizer.encode_with_chunking(texts, mode="sequential")
end = time.time()
print(f"Time taken for sequential: {end - start} seconds")

Time taken for sequential: 0.0461881160736084 seconds


In [10]:
from magicab.etoken import chunk_text, _encode_chunks



Time taken for sequential: 0.004302263259887695 seconds


In [ ]:
# Debug on pre-processing functional 
from data.composio.process_pt_data import process_fineweb_edu, process_dataset, load_from_disk
import os
dataset_config = ('fineweb-edu', process_fineweb_edu)

# ------------------- input ---------------------------
datasets_dir = "data/composio"
dataset_name, processor_fn = dataset_config
block_size=512
num_proc = 1
dataset = load_from_disk(os.path.join(datasets_dir, dataset_name))
val_size = 10
train_size = len(dataset) - val_size
train_dataset, val_dataset = process_dataset(
    dataset=dataset,
    processor_fn=processor_fn,
    tokenizer=tokenizer,
    block_size=block_size,
    num_proc=num_proc,
    train_size=train_size,
    val_size=val_size,
    desc_prefix=dataset_name
)

Processed fineweb-edu: 190 train, 10 val


In [3]:
# test with 'list-of-list' ids save & load (with padding)

all_train_ids = [[1]*600, [2]*800, [3]*100]

# 1. save to .bin 
import torch 
train_path = "test.bin"
torch.save(all_train_ids, train_path)

# 2. get_batch (load from .bin, in-sample slice with padding)


In [61]:
# Example usage
from magicab.data import get_batch_slice, save_sequences_for_memmap

text_list = ["I am super duper", "Hey how is it going?", "I am not super duper"]
all_train_ids = [tokenizer.encode(text) for text in text_list]

save_sequences_for_memmap(all_train_ids, "optimized_data.bin")

random_slice = get_batch_slice("optimized_data.bin", pad_token_id=tokenizer.pad_token_id, block_size=512)

In [60]:
random_slice = get_batch_slice("optimized_data.bin", pad_token_id=tokenizer.pad_token_id, block_size=8)
x, y = random_slice

input_ids = [int(i) for i in list(x.cpu().numpy())]
target_ids = [int(i) for i in list(y.cpu().numpy())]

input = tokenizer.decode(input_ids)
target = tokenizer.decode(target_ids)

print("Input: ", input)
print("Target: ", target)

Input:  am not s
Target:  m not su


In [29]:
# tok.encode("I am super duper") # bug 

from magicab.etoken import encode_bytes
self = tokenizer
text = "I am super duper"

# encoding 
ids = encode_bytes(text, self.special_tokens, self.special2idx, self.byte2idx)
ids = tokenizer.encode_id(text)
ids = tokenizer.encode(text)

# decoding 
tokenizer.decode(ids)

'I am super duper'

In [ ]:
# parallel encoding across chunks (async)
from magicab.etoken import chunk_text, _encode_chunks
chunk_size = 1024 

ids_list = [] 
for t in text: 
    chunks = chunk_text(t, chunk_size)
    ids = _encode_chunks(chunks, self, chunk_size)
    ids_list.append(ids)